In [24]:
pip install kagglehub

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install transformers datasets sentence-transformers langchain chromadb

Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
Using cached packaging-24.2-py3-none-any.whl (65 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 10.2 MB/s eta 0:00:00
Using cached shellingham-1.5.4-py2.py3-none-any.whl (9.8 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled pack

In [13]:
pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install --upgrade "numpy>=2.0" "scipy>=1.14" scikit-learn transformers sentence-transformers chromadb


Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-2.3.3-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 993.5 kB/s eta 0:00:00
Using cached numpy-2.3.3-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (16.9 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 50.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.6/486.6 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 47.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.3/563.3 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 44.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 38.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing in

In [61]:
import kagglehub
import json
import os
import numpy as np
import pandas as pd
import json
import re
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm  


In [2]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

In [3]:
# Download QASPER dataset using KaggleHub
dataset_path = kagglehub.dataset_download("thedevastator/qasper-nlp-questions-and-evidence")
print(f"Path to QASPER dataset: {dataset_path}")


Path to QASPER dataset: /home/dswdfb/.cache/kagglehub/datasets/thedevastator/qasper-nlp-questions-and-evidence/versions/2


In [4]:
# Download HotpotQA dataset using KaggleHub
path = kagglehub.dataset_download("jeromeblanchet/hotpotqa-question-answering-dataset")
print(f"Path to HotpotQA dataset: {path}")


Path to HotpotQA dataset: /home/dswdfb/.cache/kagglehub/datasets/jeromeblanchet/hotpotqa-question-answering-dataset/versions/1


In [30]:
import pandas as pd
import json

test = pd.read_csv(f"{dataset_path}/test.csv")
train = pd.read_csv(f"{dataset_path}/train.csv")
validation = pd.read_csv(f"{dataset_path}/validation.csv")



We had issues with the Keggle CSV file. Tried parsing it manually as well as using the nested objects, but it still gave errors. This may be because of the object is converted to a string and the string contains Object types like 'Array()' which cause the json parse error.

We are using the same data sets but via HuggingFace, which provides straightfaward json object to parse

### Load the QASPER from huggingface

In [48]:
qasper_ds = load_dataset("allenai/qasper", split="train")

In [49]:
rows = []
for paper in qasper_ds:
    paper_id = paper["id"]
    title    = paper["title"]
    abstract = paper["abstract"]

    # full_text is a dict of columns
    sec_names = paper["full_text"]["section_name"]
    sec_paras = paper["full_text"]["paragraphs"]
    full_text = "\n\n".join(
        f"{sec}\n" + "\n".join(p) for sec, p in zip(sec_names, sec_paras)
    )

    qas = paper["qas"]
    n_questions = len(qas["question"])

    for i in range(n_questions):
        question_id   = qas["question_id"][i]
        question_text = qas["question"][i]
        nlp_bg        = qas["nlp_background"][i]
        topic_bg      = qas["topic_background"][i]
        paper_read    = qas["paper_read"][i]
        search_query  = qas["search_query"][i]
        question_writer = qas["question_writer"][i]

        # answers is ALSO a dict of parallel lists
        answers_block = qas["answers"][i]
        for ans, ann_id, worker_id in zip(
            answers_block["answer"],
            answers_block["annotation_id"],
            answers_block["worker_id"]
        ):
            rows.append({
                "paper_id"        : paper_id,
                "title"           : title,
                "abstract"        : abstract,
                "full_text"       : full_text,
                "question_id"     : question_id,
                "question"        : question_text,
                "nlp_background"  : nlp_bg,
                "topic_background": topic_bg,
                "paper_read"      : paper_read,
                "search_query"    : search_query,
                "question_writer" : question_writer,
                "annotation_id"   : ann_id,
                "worker_id"       : worker_id,
                "unanswerable"    : ans["unanswerable"],
                "yes_no"          : ans["yes_no"],
                "free_form_answer": ans["free_form_answer"],
                "extractive_spans": "; ".join(ans["extractive_spans"]),
                "evidence"        : "; ".join(ans["evidence"]),
                "highlighted_evidence": "; ".join(ans["highlighted_evidence"])
            })

qasper_df = pd.DataFrame(rows)

In [69]:
qasper_df.to_csv('processed_qasper_data.csv', index=False)

In [50]:
qasper_df.head()

,paper_id,title,abstract,full_text,question_id,question,nlp_background,topic_background,paper_read,search_query,question_writer,annotation_id,worker_id,unanswerable,yes_no,free_form_answer,extractive_spans,evidence,highlighted_evidence
0,1909.00694,Minimally Supervised Learning of Affective Eve...,Recognizing affective events that trigger posi...,Introduction\nAffective events BIBREF0 are eve...,753990d0b621d390ed58f20c4d9e4f065f0dc672,What is the seed lexicon?,two,unfamiliar,no,,c1fbdd7a261021041f75fbe00a55b4c386ebbbb4,31e85022a847f37c15fd0415f3c450c74c8e4755,c1fbdd7a261021041f75fbe00a55b4c386ebbbb4,False,None,a vocabulary of positive and negative predicat...,,The seed lexicon consists of positive and nega...,The seed lexicon consists of positive and nega...
1,1909.00694,Minimally Supervised Learning of Affective Eve...,Recognizing affective events that trigger posi...,Introduction\nAffective events BIBREF0 are eve...,753990d0b621d390ed58f20c4d9e4f065f0dc672,What is the seed lexicon?,two,unfamiliar,no,,c1fbdd7a261021041f75fbe00a55b4c386ebbbb4,95da0a6e1b08db74a405c6a71067c9b272a50ff5,2cfd959e433f290bb50b55722370f0d22fe090b7,False,None,,seed lexicon consists of positive and negative...,The seed lexicon consists of positive and nega...,The seed lexicon consists of positive and nega...
2,1909.00694,Minimally Supervised Learning of Affective Eve...,Recognizing affective events that trigger posi...,Introduction\nAffective events BIBREF0 are eve...,9d578ddccc27dd849244d632dd0f6bf27348ad81,What are the results?,two,unfamiliar,no,,c1fbdd7a261021041f75fbe00a55b4c386ebbbb4,1e5e867244ea656c4b7632628086209cf9bae5fa,2cfd959e433f290bb50b55722370f0d22fe090b7,False,None,Using all data to train: AL -- BiGRU achieved ...,,FLOAT SELECTED: Table 3: Performance of variou...,FLOAT SELECTED: Table 3: Performance of variou...
3,1909.00694,Minimally Supervised Learning of Affective Eve...,Recognizing affective events that trigger posi...,Introduction\nAffective events BIBREF0 are eve...,02e4bf719b1a504e385c35c6186742e720bcb281,How are relations used to propagate polarity?,two,unfamiliar,no,,c1fbdd7a261021041f75fbe00a55b4c386ebbbb4,49a78a07d2eed545556a835ccf2eb40e5eee9801,c1fbdd7a261021041f75fbe00a55b4c386ebbbb4,False,None,"based on the relation between events, the sugg...",,"In this paper, we propose a simple and effecti...","As illustrated in Figure FIGREF1, our key idea..."
4,1909.00694,Minimally Supervised Learning of Affective Eve...,Recognizing affective events that trigger posi...,Introduction\nAffective events BIBREF0 are eve...,02e4bf719b1a504e385c35c6186742e720bcb281,How are relations used to propagate polarity?,two,unfamiliar,no,,c1fbdd7a261021041f75fbe00a55b4c386ebbbb4,acd6d15bd67f4b1496ee8af1c93c33e7d59c89e1,2cfd959e433f290bb50b55722370f0d22fe090b7,False,None,cause relation: both events in the relation sh...,,"In this paper, we propose a simple and effecti...","As illustrated in Figure FIGREF1, our key idea..."


## Step 1: Summarization

### We will store context in ChromaDb with chunking, for this task we will use 256 as our chunk size

In [51]:
hf_embedding_function = SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

2025-09-25 12:33:05.346401: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758821585.367009    2224 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758821585.374990    2224 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-25 12:33:05.402834: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [70]:
chroma_client = chromadb.PersistentClient(path="./ChromaDb")
collection_name = "qasper_data_collection"
chroma_client.delete_collection(name=collection_name)

chroma_collection = chroma_client.create_collection(
    collection_name,
    embedding_function=hf_embedding_function
)

In [71]:
character_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=1000,
    chunk_overlap=10
)

In [72]:
token_splitter = SentenceTransformersTokenTextSplitter(
    chunk_overlap=10,
    tokens_per_chunk=256
)

In [73]:
for paper_id, group in tqdm(qasper_df.groupby("paper_id")):
    full_text = str(group.iloc[0]["full_text"])

    char_chunks = character_splitter.split_text(full_text)

    token_chunks = []
    for chunk in char_chunks:
        token_chunks.extend(token_splitter.split_text(chunk))

    if not token_chunks:
        print(f"Skipping paper {paper_id}: no chunks produced.")
        continue
    
    ids = [f"{paper_id}_{i}" for i in range(len(token_chunks))]
    question_ids = group["question_id"].tolist()
    metadatas = [
        {
            "paper_id": paper_id,
            "question_ids": ",".join(question_ids),
        }
        for _ in token_chunks
    ]

    chroma_collection.add(
        documents=token_chunks,
        ids=ids,
        metadatas=metadatas
    )

print("All papers processed and stored in Chroma.")


 24%|██▍       | 215/888 [01:31<03:53,  2.88it/s]

Skipping paper 1801.07804: no chunks produced.


100%|██████████| 888/888 [07:04<00:00,  2.09it/s]

All papers processed and stored in Chroma.


### We have Store all the Data For QASPER DATA SET AND WE MOVE INTO LOAD MODELS NOTEBOOK FOR ANSWER GENERATION